In [4]:
import PySimpleGUI as sg
from lyricsgenius import Genius
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
import text2emotion
from nrclex import NRCLex
import googletrans
from googletrans import Translator
import langdetect
from langdetect import detect

headerT2E = ['Lyric','Happy','Angry','Suprise','Sad','Fear']
headerNRC = ['Lyric','Fear','Anger','Anticipation','Trust','Suprise','Sadness','Disgust','Joy']

def make_win1():
    column1 = [
        [
            sg.Text('Select analyzer: ')
        ],
        [
            sg.Combo(['Text2Emotion', 'NRCLex'], enable_events=True, key='combo', default_value='Text2Emotion')
        ]
    ]
    column2 = [
        [
            sg.Button("Analyse", enable_events=True, size=(10,1), key="analyse")
        ]
    ]
    
    layout = [
        [
            sg.Text("Song"),
            sg.In(size=(25, 1), enable_events=True, key="song"),
            sg.Button("Search",enable_events=True, key="plus"),
        ],
        [
            sg.Listbox(
                values=[], enable_events=True, size=(40, 20), key="songList"
            )
        ],
        [
            sg.Column(column1),
            sg.VSeperator(),
            sg.Column(column2),
        ]
    ]
    return sg.Window('MusicEmotioner', layout, finalize=True, element_justification='c')

def make_win2():

    layout = [
        [
            sg.Text("Song: "), 
            sg.In(size=(25, 1), key="song", readonly=True)
        ],
        [
            sg.Text("Happy: "), 
            sg.In(size=(10, 1), key="gHappy", readonly=True),
            sg.Text("Angry: "), 
            sg.In(size=(10, 1), key="gAngry", readonly=True),
            sg.Text("Suprise: "), 
            sg.In(size=(10, 1), key="gSurp", readonly=True),
            sg.Text("Sad: "), 
            sg.In(size=(10, 1), key="gSad", readonly=True),
            sg.Text("Fear: "), 
            sg.In(size=(10, 1), key="gFear", readonly=True)
        ],
        [
            sg.Table(values=[[]], auto_size_columns=False, col_widths=[40,10,10,10,10,10],
                     display_row_numbers=True, key='table', enable_events=True,
                     headings=headerT2E, justification='center')
        ]
    ]

    return sg.Window('text2emotion', layout, finalize=True)

def make_win3():
    layout = [
        [
            sg.Text("Song: "), 
            sg.In(size=(25, 1), key="song", readonly=True)
        ],
        [
            sg.Text("Fear: "), 
            sg.In(size=(9, 1), key="gFear", readonly=True),
            sg.Text("Anger: "), 
            sg.In(size=(9, 1), key="gAnger", readonly=True),
            sg.Text("Anticipation: "), 
            sg.In(size=(9, 1), key="gAnt", readonly=True),
            sg.Text("Trust: "), 
            sg.In(size=(9, 1), key="gTrust", readonly=True),
            sg.Text("Suprise: "), 
            sg.In(size=(9, 1), key="gSurp", readonly=True),
            sg.Text("Sadness: "), 
            sg.In(size=(9, 1), key="gSad", readonly=True),
            sg.Text("Disgust: "), 
            sg.In(size=(9, 1), key="gDisg", readonly=True),
            sg.Text("Joy: "), 
            sg.In(size=(9, 1), key="gJoy", readonly=True)
        ],
        [
            sg.Table(values=[[]], auto_size_columns=False, col_widths=[40,10,10,10,10,10],
                     display_row_numbers=True, key='table', enable_events=True,
                     headings=headerNRC, justification='center')
        ]
    ]
    
    return sg.Window('NRCLex', layout, finalize=True)

def t2eAnalysis(window, values):
    data = values["songList"][0].split(' by ')
    failure = True
    genius = Genius("0UQaYZMpHpBO8ZM1_tvKODTw1U2hyN3z309kh1h-X7wqyjn6Ydm5PUHqrWjY5jFN")
    while failure:
        try:
            song = genius.search_song(data[0], data[1])
            failure = False
        except:
            pass

    sents = song.lyrics.splitlines()

    sents.pop(0)
    sents.pop(0)

    lang=detect(sents[0])
    if lang!='en':
        translator = Translator()
        sents = [translator.translate(sent).text for sent in sents]

    lyrics = " ".join(sents)
    genEmotion = text2emotion.get_emotion(lyrics)
    
    lyEmotions = []
    for sent in sents:
        if not sent.startswith('[') and someWord(sent):
            lyEmotions.append([sent]+[str(em) for em in list(text2emotion.get_emotion(sent).values())])

    window["song"].update(values["songList"][0])

    window["gHappy"].update(str(round(genEmotion['Happy']*100))+'%')
    window["gAngry"].update(str(round(genEmotion["Angry"]*100))+'%')
    window["gSurp"].update(str(round(genEmotion["Surprise"]*100))+'%')
    window["gSad"].update(str(round(genEmotion["Sad"]*100))+'%')
    window["gFear"].update(str(round(genEmotion["Fear"]*100))+'%')

    window["table"].update(lyEmotions)
    
def nrcAnalysis(window, values):    
    data = values["songList"][0].split(' by ')
    
    failure = True
    genius = Genius("0UQaYZMpHpBO8ZM1_tvKODTw1U2hyN3z309kh1h-X7wqyjn6Ydm5PUHqrWjY5jFN")
    while failure:
        try:
            song = genius.search_song(data[0], data[1])
            failure = False
        except:
            pass    
    
    sents = song.lyrics.splitlines()

    sents.pop(0)
    sents.pop(0)

    lang=detect(sents[0])
    if lang!='en':
        translator = Translator()
        sents = [translator.translate(sent).text for sent in sents]

    lyrics = " ".join(sents)
    emotion = NRCLex(lyrics)
    genEmotion = emotion.affect_frequencies
    
    lyEmotions = []
    for sent in sents:
        if not sent.startswith('[') and someWord(sent):
            emotion = NRCLex(sent)
            lyEmotions.append([sent]+[str(em) for em in list(emotion.affect_frequencies.values())])
    
    window["song"].update(values["songList"][0])

    window["gFear"].update(str(round(genEmotion['fear']*100,3))+'%')
    window["gAnger"].update(str(round(genEmotion["anger"]*100,3))+'%')
    window["gAnt"].update(str(round(genEmotion["anticipation"]*100,3))+'%')
    window["gTrust"].update(str(round(genEmotion["trust"]*100,3))+'%')
    window["gSurp"].update(str(round(genEmotion["surprise"]*100,3))+'%')
    window["gSad"].update(str(round(genEmotion["sadness"]*100,3))+'%')
    window["gDisg"].update(str(round(genEmotion["disgust"]*100,3))+'%')
    window["gJoy"].update(str(round(genEmotion["joy"]*100,3))+'%')

    window["table"].update(lyEmotions)

def someWord(sent):
    words = word_tokenize(sent)
    for word in words:
        if len(wn.synsets(word))>0:
            return True
    return False

#other = [
#    [
#        sg.Column(file_list_column, size=(300,390)),
#        sg.VSeperator(),
#        sg.Column(image_viewer_column, size=(500,390)),
#    ]
#]

In [6]:
def musicAnalyzer():
    window1 = make_win1()
    window2 = None
    window3 = None

    # Create an event loop
    while True:
        genius = Genius("0UQaYZMpHpBO8ZM1_tvKODTw1U2hyN3z309kh1h-X7wqyjn6Ydm5PUHqrWjY5jFN")
        window, event, values = sg.read_all_windows()
        if event == "Exit" or event == sg.WIN_CLOSED:
            if window == window2:
                window2.close()
            elif window == window3:
                window3.close()
            else:
                break

        elif event == "plus":
            title = values["song"]

            failure = True
            while failure:
                try:
                    songs = genius.search_songs(title)
                    failure = False
                except:
                    pass

            snames = [
                s['result']['title'] +" by "+ s['result']['artist_names']
                for s in songs["hits"]
            ]

            window["songList"].update(snames)

        elif event == "analyse":
            if values['combo']=='Text2Emotion':
                window2 = make_win2()

                t2eAnalysis(window2, values)
            elif values['combo']=='NRCLex':
                window3 = make_win3()
                
                nrcAnalysis(window3, values)

    window1.close()

In [ ]:
musicAnalyzer()